In [1]:
import numpy as np
import torch as T
import scipy; import scipy.optimize;
import time;

import matplotlib;
import matplotlib.pyplot as plt

import  forward_method as FM;


In [37]:
"""
cell 1: model of 
define distance between field and reflector;
define the smapling points.
"""
D=100*10**3; # distance between source and reflector
Nu=Nv=11;   #sampling points of far-/near-field
noise=0.5;  # rms of the noise added to measurement pattern.
rms_surface=0.08#mm # rms of the surface error.

Nx=5;    # sampling poitns on x direction; for panels
Ny=5;    # sampling points on y direction;
##############################################################################
List=np.array([[-500,500,-1500,-500,500,1500,-1500,-500,500,1500,-500,500],
               [1500,1500,500,500,500,500,-500,-500,-500,-500,-1500,-1500]]);
size=1000; # size of panel
Nx=5;    # sampling poitns on x direction;
Ny=5;    # sampling points on y direction;
f=100*10**3;

Lambda=1#mm
sigma=1200;Amp_0=1
# define u v range

u=np.linspace(-1,1,Nu)*0.002;
v=np.linspace(-1,1,Nu)*0.002;
gird=np.moveaxis(np.meshgrid(u,v),0,-1);
u=gird[...,0];
v=gird[...,1];
m_x,m_y,m_z=FM.reflector(List,size,Nx,Ny,f)
far_field=FM.get_function(List,size,Nx,Ny,f,u,v,f,Lambda,sigma,Amp_0);

# near field
near_field=FM.get_function(List,size,Nx,Ny,f,u,v,D,Lambda,sigma,Amp_0);

In [38]:
"""
Cell 2: get the measurement data!
"""
# get measurement data;

ad1=np.random.normal(0,rms_surface,(3,12));

p=T.tensor(FM.adjuster2P(ad1));

real_meas,imag_meas=near_field(p);

if noise != 0:
    FM.addnoise(real_meas,imag_meas,0,noise);# add noise with measurement data;


Data_meas=T.cat((real_meas,imag_meas));


In [43]:
"""
Cell 3:
fitting process with unit_vectors
"""

start=time.clock();
ad=np.zeros((3,12));
dz=10/1000;
for n in range(5):
    reference,base_vectors=FM.basevectors(near_field,ad,dz);
    test=Data_meas-reference;
    def fitfn(p):
        
        #p=T.tensor(p, requires_grad=True);        
        p=T.tensor(p).view(-1,1);
        r=(test-T.sum(p*base_vectors,0));
        return r.data.cpu().numpy()#,p.grad.data.cpu().numpy()
    para=np.zeros((3,12));
    x=scipy.optimize.leastsq(fitfn,para);
    #print(x[0])
    ad=ad+x[0].reshape((3,12))*dz;
    dz=1/1000;
    
elapsed = (time.clock() - start);
print("Time used:",elapsed);   
error=np.sum(np.abs(ad-ad1))/36*1000;
print('1. fitting error: %e um'% error);
print('2. fitting results:');
print(ad);
print('3. real adjusters of test:')
print(ad1);

    

Time used: 0.47502782085757644
1. fitting error: 1.079385e+01 um
2. fitting results:
[[-0.07414452 -0.06975436 -0.02578214 -0.13186334 -0.02720469  0.01871247
   0.03011735  0.00892088 -0.09734205  0.04793119  0.01538287  0.02072944]
 [-0.09455988 -0.13108293  0.0427587  -0.03007558 -0.04842263  0.14204247
   0.00446034 -0.0884737  -0.0086313  -0.06358072 -0.08700857  0.05323603]
 [-0.15341208 -0.07200725 -0.00459877  0.08444798 -0.01811255 -0.03694316
  -0.0150464  -0.06146558  0.01765128 -0.13353758  0.09802521 -0.20914865]]
3. real adjusters of test:
[[-0.07286606 -0.07347146 -0.04582549 -0.13442543 -0.02161806  0.01876775
   0.0102924   0.00990572 -0.0959331   0.02420154  0.03397087  0.02430042]
 [-0.10695027 -0.13705459  0.05671959 -0.02681405 -0.04272744  0.12357636
   0.04779676 -0.09246239 -0.00196538 -0.10406666 -0.07648266  0.05594691]
 [-0.13102039 -0.07553561 -0.00869023  0.07426502 -0.01635036 -0.02412607
  -0.02771401 -0.06000308  0.01798137 -0.10877225  0.07871252 -0.215

In [31]:
print(ad1*1000);

[[ 103.27379625    5.38956876   54.11663722  248.71289249   49.02074771
   -22.40527296   95.26638149   21.77350913   71.52406873   55.54901505
   -11.73040511  -63.1247828 ]
 [  85.51284788   17.76524264  147.45809605 -125.1252043   131.73737673
   135.65248326  -15.1553304   -52.96311081  113.98851052  101.85027403
  -120.50340031   -0.92187639]
 [  77.30897889  -62.47802762   -8.72353094  -25.37890197   46.94494768
    -1.67079341    0.60782217 -102.86611635   26.92790011   -1.82389603
   -81.70899599  -72.61132167]]
